In [36]:
import re
from settings import *

text = "\n   aa   ezigko zez\n\nef\n j •j •  j.  dz...  '$;.  \n<eom><eod>\n\n<eom>  \t  %kzeé izçr zg    ¨e:=zge \ng   ezù =   = gez $gez  \n"

text = text.replace('\n', '<nl>').replace('\t', '    ')

# Start

# Split control characters

words = [text]
temp = []

for control in CONTROL_CHARS:
	for i in range(len(words)):

		parts = words[i].split(control)

		for part in parts:
			if len(part) > 0:
				temp.append(part)

			temp.append(control)

		if len(parts) > 0:
			temp.pop()

	words = temp
	temp = []

# Split words

for i in range(len(words)):

	if words[i] in CONTROL_CHARS:
		temp.append(words[i])
		continue

	word = ''

	for j in range(len(words[i])):

		if len(word) == 0:
			word += words[i][j]
			continue

		if word[-1] != ' ' and words[i][j] == ' ':
			temp.append(word)
			word = words[i][j]
			continue

		if word[-1].isalpha() and not words[i][j].isalpha():
			temp.append(word)
			word = words[i][j]
			continue

		if word[-1] != ' ' and not word[-1].isalpha() and words[i][j].isalpha():
			temp.append(word)
			word = words[i][j]
			continue

		word += words[i][j]

	if len(word) > 0:
		temp.append(word)

words = temp
temp = []

# Split multiple spaces

for i in range(len(words)):

	j = 0

	while j < len(words[i]) and words[i][j] == ' ':
		j += 1

	if j > 1 and j < len(words[i]):
		temp.append(words[i][:j - 1])
		temp.append(words[i][j - 1:])

	else:
		temp.append(words[i])

words = temp
temp = []

# Stop

print(text.replace(' ', '_'))

for i in range(len(words)):
	if len(words[i]) == 0:
		print('\033[92m_\033[0m', end = '')
	if i % 2 == 1:
		print('\033[91m' + words[i].replace(' ', '_') + '\033[0m', end = '')
	else:
		print('\033[94m' + words[i].replace(' ', '_') + '\033[0m', end = '')

<nl>___aa___ezigko_zez<nl><nl>ef<nl>_j_•j_•__j.__dz...__'$;.__<nl><eom><eod><nl><nl><eom>________%kzeé_izçr_zg____¨e:=zge_<nl>g___ezù_=___=_gez_$gez__<nl>
<nl>___aa___ezigko_zez<nl><nl>ef<nl>_j_•j_•__j.__dz...__'$;.__<nl><eom><eod><nl><nl><eom>________%kzeé_izçr_zg____¨e:=zge_<nl>g___ezù_=___=_gez_$gez__<nl>

In [ ]:
print('a.n'.split('.'))